In [3]:
import xml.etree.ElementTree as ET
import pandas as pd
import json
import os
import great_expectations as ge

In [4]:
ITERATOR_PATH = ".//carrier/*"
MULTISELECT_PATH = "multiselect"

In [5]:
def process_xml(file):
    tree = ET.parse(file)
    root = tree.getroot()
    
    record = {}
    for el in root.findall(ITERATOR_PATH):
        if len(el) > 0:
            record[el.tag] = ';'.join(list(map(lambda x: x.text, el.findall(MULTISELECT_PATH))))
        else:
            record[el.tag] = el.text
            
    return record

In [6]:
xml_dir = "./data"
listofxml = os.listdir(xml_dir)

records = [ process_xml(xml_dir +'/'+ xml) for xml in listofxml ]
df = pd.DataFrame.from_records(records)

In [7]:
ge_df = ge.from_pandas(df)
ge_df.head()

,CP,CP_id,Sub_CP,PID,created_on,created_by,updated_on,updated_by,type,gauge,...,remarks,donor_of_the_archive,record_status,collection_box_barcode,batch_name,batch_id,batch_pickup_date,file_duration,transfer_lto_date,lto_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CP,OR-xxxx,NaN,1j9765cj05,2015-08-20T16:08:08,voornaam achternaam,2016-09-08T11:09:09,voornaam achternaam,film,16mm,...,NaN,NaN,reviewed by VIAA,NaN,FLMB16,FLMB16,2019-11-20,NaN,NaN,IM0218L5


In [ ]:
ge_df.expect_column_values_to_be_unique(column="CP")

In [ ]:
import json

with open( "xml_test_expectation.json", "w") as my_file:
    my_file.write(
        json.dumps(my_df.get_expectation_suite().to_json_dict())
    )